In [1]:
import json

import numpy as np
from chainer.datasets import TupleDataset, split_dataset_random
from chainer.iterators import SerialIterator
from PIL import Image

/Users/hassaku/.pyenv/versions/3.7.0/lib/python3.7/site-packages/chainer/_environment_check.py:37: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BLAS libraries such as OpenBLAS.
For details of the issue, please see
https://docs.chainer.org/en/stable/tips.html#mnist-example-does-not-converge-in-cpu-mode-on-mac-os-x.

Please be aware that Mac OS X is not an officially supported OS.

  ''')  # NOQA


In [58]:
class DataLoader():
    def __init__(self, annotation_file_path):
        self.height = 224
        self.width = 224
        self.channel = 3
        self.label_num = 16
        self.annotation_file_path = annotation_file_path
        self.annotation_file = json.load(open((self.annotation_file_path), 'r'))
        self.image_list = self.annotation_file['fileList']
        self.label_list = self.annotation_file['posList']

    def load_data(self):
        x = np.zeros((len(self.image_list), self.channel, self.height, self.width), dtype='float32')
        for i, imagepath in enumerate(self.image_list):
            image = Image.open(imagepath).convert('RGB')
            np_image = np.asarray(image.resize((self.height, self.width)))
            x[i] = np.moveaxis(np_image, 2, 0)
        self.x = x

        t = np.zeros((len(self.label_list), self.label_num, self.height, self.width), dtype='float32')
        for label_idx, label in enumerate(self.label_list):
            for point_idx, point in enumerate(label):
                x_pos = int(point[0] / 600 * 56)
                y_pos = int(point[1] / 600 * 56)
                t[label_idx][point_idx][y_pos][x_pos] = 1
        self.t = t

        self.dataset = TupleDataset(x, t)
        return self.dataset
        
    def split(self, ratio = (8, 1, 1)):
        train_val_rate = (ratio[0] + ratio[1]) / sum(ratio)
        train_rate = ratio[0] / (ratio[0] + ratio[1])
        train_val, test = split_dataset_random(self.dataset, int(len(self.dataset) * train_val_rate), seed=0)
        train, valid = split_dataset_random(train_val, int(len(train_val) * train_rate), seed=0)
        self.train = train
        self.valid = valid
        self.test = test
        return (self.train, self.valid, self.test)

In [59]:
dataloader = DataLoader('../data/raw/hands/hand_position.json')
dataloader.load_data()
train, valid, test = dataloader.split((8, 1, 1))

In [94]:
import chainer.links as L
import chainer.functions as F

class network():
    def __init__(self):
        self.base = L.VGG16Layers()
        self.dc1 = L.Deconvolution2D(512, 128) # in 7x7, out 14x14
        self.dc2 = L.Deconvolution2D(128, 32) # in 14x14, out 28x28
        self.dc3 = L.Deconvolution2D(32, 16) # in 28x28, out 56x56
        
        
    def __call__(self, x):
        h = self.base(x, layers=['pool5'])
        h = self.dc1(h['pool5'])
        h = self.dc2(h)
        h = self.dc3(h)
        print(h.shape)
        return y

In [95]:
# gpu_id = 0  # 使用する GPU 番号
n_batch = 2  # バッチサイズ
n_epoch = 50  # エポック数

# ネットワークを GPU メモリ上に転送
# net.to_gpu(gpu_id)

# ログ
results_train, results_valid = {}, {}
results_train['loss'], results_train['accuracy'] = [], []
results_valid['loss'], results_valid['accuracy'] = [], []

count = 1

iteration = 0

# ログの保存用
results_train = {
    'loss': [],
    'accuracy': []
}
results_valid = {
    'loss': [],
    'accuracy': []
}

In [96]:
from chainer.iterators import SerialIterator
train_iter = SerialIterator(train, batch_size=4, repeat=True, shuffle=True)

In [ ]:
from chainer.dataset import concat_examples
net = network()
for epoch in range(n_epoch):
    while True:
        # ミニバッチの取得
        train_batch = train_iter.next()
        # x と t に分割
        # データを GPU に転送するために、concat_examples に gpu_id を渡す
        x_train, t_train = concat_examples(train_batch) #, gpu_id)

        # 予測値と目的関数の計算
        print('Calc y_train.')
        print(x_train.shape)
        y_train = net(x_train)
        print(y_train.shape)
        loss_train = F.mean_squared_error(y_train, t_train)
        acc_train = F.accuracy(y_train, t_train)

        # 勾配の初期化と勾配の計算
        net.cleargrads()
        loss_train.backward()

        # パラメータの更新
        optimizer.update()

        # カウントアップ
        count += 1

        # 1エポック終えたら、valid データで評価する
        if train_iter.is_new_epoch:
            print('epoch', count, 'end.')

Calc y_train.
(4, 3, 224, 224)
